# MAST30034 Project 1
## Preprocessing Weather

In [1]:
import pandas as pd

LaGuardia Airport was chosen as the area taken for weather measurements as out of the 3 major airports in NYC, it is the most centrally located and thus will be the best estimate of the overal weather across the city.

In [2]:
LG18 = pd.read_csv("../../Project 1/Data/Weather Data/LaGuardia2018.csv")
df = pd.read_csv("../../Project 1/Data/Weather Data/LaGuardia2019.csv")
print(len(LG18.columns))
print(len(df.columns))

100
102


/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_20969/3527527484.py:1: DtypeWarning: Columns (44,45,58,89) have mixed types. Specify dtype option on import or set low_memory=False.
  LG18 = pd.read_csv("../../Project 1/Data/Weather Data/LaGuardia2018.csv")
/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_20969/3527527484.py:2: DtypeWarning: Columns (44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../Project 1/Data/Weather Data/LaGuardia2019.csv")


In [3]:
df = df.append(LG18)

/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_20969/3565837567.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(LG18)


The datasets have approx 100 columns with highly specific weather for a variety of needs, as this project is looking at how weather and other variables effect tipping in NYC, the variables chosen were the main distinct variables commonly reported on: Wind Direction, Wind Speed, Temperature, Dew Point, Sea Level Pressure

In [4]:
df = df[['DATE', 'WND', 'TMP', 'DEW', 'SLP']]
df = df.reset_index()


Cleaning up the data

In [5]:
import numpy as np

df['date']=df['DATE'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df['wind_speed']=df['WND'].str.extract(r'\d{3},\d,\w,(\d{4},\d)')
df['wind_direction']=df['WND'].str.extract(r'(\d{3},\d),\w,\d{4},\d')
df = df.rename(columns={"TMP":"temp", "DEW":"dew_point", "SLP":"pressure"})
df['wind_direction'] = df['wind_direction'].str.replace(',','.',regex=True)
df['wind_speed'] = df['wind_speed'].str.replace(',','.',regex=True)
df['temp'] = df['temp'].str.replace(',','.',regex=True)
df['dew_point'] = df['dew_point'].str.replace(',','.',regex=True)
df['pressure'] = df['pressure'].str.replace(',','.',regex=True)
df = df.drop(['DATE', 'WND'], axis =1)
df


,index,temp,dew_point,pressure,date,wind_speed,wind_direction
0,0,+0067.1,+0050.1,10201.1,2019-01-01,0041.1,130.1
1,1,+0070.5,+0050.5,99999.9,2019-01-01,0067.5,140.5
2,2,+0067.5,+0050.5,10181.5,2019-01-01,0067.5,130.5
3,3,+0072.5,+0056.5,99999.9,2019-01-01,0046.5,140.5
4,4,+0072.5,+0056.5,99999.9,2019-01-01,0051.5,130.5
...,...,...,...,...,...,...,...
28356,14275,+0067.5,+0033.5,10219.5,2018-12-31,0036.5,130.5
28357,14276,+0067.5,+0044.5,10211.5,2018-12-31,0046.5,120.5
28358,14277,+0067.5,+0044.5,99999.9,2018-12-31,0041.5,130.5
28359,14278,+0067.5,+0050.5,99999.9,2018-12-31,0046.5,130.5


Converting Strings to Floats:

In [6]:
df = df[df["temp"].str.contains("A")==False]
df['pressure'] = pd.to_numeric(df['pressure'], downcast='float')
df['temp'] = pd.to_numeric(df['temp'], downcast='float')
df['dew_point'] = pd.to_numeric(df['dew_point'], downcast='float')
df['wind_speed'] = pd.to_numeric(df['wind_speed'], downcast='float')
df['wind_direction'] = pd.to_numeric(df['wind_direction'], downcast='float')
df = df.drop('index', axis=1)


/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_20969/120665663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pressure'] = pd.to_numeric(df['pressure'], downcast='float')
/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_20969/120665663.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp'] = pd.to_numeric(df['temp'], downcast='float')
/var/folders/ck/psrltvd10474n3pnlfszjw500000gn/T/ipykernel_20969/120665663.py:4: SettingWithCopyWarning: 
A value is trying to be set on

### Invalidating data:
#### Temp
Maximum recorded temperature in NYC was 106F and the Mminium was -15F, thus these were set as the bounds

In [7]:
df = df.drop(df[(df.temp > 106)].index)
df = df.drop(df[(df.temp < -15)].index)
df

,temp,dew_point,pressure,date,wind_speed,wind_direction
0,67.099998,50.099998,10201.099609,2019-01-01,41.099998,130.100006
1,70.500000,50.500000,99999.898438,2019-01-01,67.500000,140.500000
2,67.500000,50.500000,10181.500000,2019-01-01,67.500000,130.500000
3,72.500000,56.500000,99999.898438,2019-01-01,46.500000,140.500000
4,72.500000,56.500000,99999.898438,2019-01-01,51.500000,130.500000
...,...,...,...,...,...,...
28356,67.500000,33.500000,10219.500000,2018-12-31,36.500000,130.500000
28357,67.500000,44.500000,10211.500000,2018-12-31,46.500000,120.500000
28358,67.500000,44.500000,99999.898438,2018-12-31,41.500000,130.500000
28359,67.500000,50.500000,99999.898438,2018-12-31,46.500000,130.500000


#### Dew Point
Maximum recorded dew point in NYC was 106F and the Mminium was -49F, thus these were set as the bounds

In [8]:
df = df.drop(df[(df.dew_point > 106)].index)
df = df.drop(df[(df.dew_point < -49)].index)
df

,temp,dew_point,pressure,date,wind_speed,wind_direction
0,67.099998,50.099998,10201.099609,2019-01-01,41.099998,130.100006
1,70.500000,50.500000,99999.898438,2019-01-01,67.500000,140.500000
2,67.500000,50.500000,10181.500000,2019-01-01,67.500000,130.500000
3,72.500000,56.500000,99999.898438,2019-01-01,46.500000,140.500000
4,72.500000,56.500000,99999.898438,2019-01-01,51.500000,130.500000
...,...,...,...,...,...,...
28356,67.500000,33.500000,10219.500000,2018-12-31,36.500000,130.500000
28357,67.500000,44.500000,10211.500000,2018-12-31,46.500000,120.500000
28358,67.500000,44.500000,99999.898438,2018-12-31,41.500000,130.500000
28359,67.500000,50.500000,99999.898438,2018-12-31,46.500000,130.500000


#### Pressure
There isnt much data for the max and min sea level pressure of NYC, so the upper bounds were set to be the worlds max and min to be safe.

In [9]:
df = df.drop(df[df.pressure > 10600].index)
df = df.drop(df[df.pressure < 8700].index)
df

,temp,dew_point,pressure,date,wind_speed,wind_direction
0,67.099998,50.099998,10201.099609,2019-01-01,41.099998,130.100006
2,67.500000,50.500000,10181.500000,2019-01-01,67.500000,130.500000
6,72.500000,61.500000,10164.500000,2019-01-01,46.500000,120.500000
9,78.500000,61.500000,10139.500000,2019-01-01,62.500000,120.500000
10,78.099998,61.099998,10139.099609,2019-01-01,62.099998,120.099998
...,...,...,...,...,...,...
28354,67.500000,33.500000,10235.500000,2018-12-31,31.500000,150.500000
28355,67.099998,33.099998,10235.099609,2018-12-31,31.100000,150.100006
28356,67.500000,33.500000,10219.500000,2018-12-31,36.500000,130.500000
28357,67.500000,44.500000,10211.500000,2018-12-31,46.500000,120.500000


#### Wind Speed 
highest ever record was 185km/h during hurricane sandy  and ofcourse the slowest is set to 0.

In [10]:
df = df.drop(df[df.wind_speed > 185].index)
df = df.drop(df[df.wind_speed < 0].index)
df

,temp,dew_point,pressure,date,wind_speed,wind_direction
0,67.099998,50.099998,10201.099609,2019-01-01,41.099998,130.100006
2,67.500000,50.500000,10181.500000,2019-01-01,67.500000,130.500000
6,72.500000,61.500000,10164.500000,2019-01-01,46.500000,120.500000
9,78.500000,61.500000,10139.500000,2019-01-01,62.500000,120.500000
10,78.099998,61.099998,10139.099609,2019-01-01,62.099998,120.099998
...,...,...,...,...,...,...
28354,67.500000,33.500000,10235.500000,2018-12-31,31.500000,150.500000
28355,67.099998,33.099998,10235.099609,2018-12-31,31.100000,150.100006
28356,67.500000,33.500000,10219.500000,2018-12-31,36.500000,130.500000
28357,67.500000,44.500000,10211.500000,2018-12-31,46.500000,120.500000


#### Wind Direction 
wind direction is set in relation to compass degrees and thus the max is 360 and min is 0:

In [11]:
df = df.drop(df[df.wind_direction > 360].index)
df = df.drop(df[df.wind_direction < 0].index)
df

,temp,dew_point,pressure,date,wind_speed,wind_direction
0,67.099998,50.099998,10201.099609,2019-01-01,41.099998,130.100006
2,67.500000,50.500000,10181.500000,2019-01-01,67.500000,130.500000
6,72.500000,61.500000,10164.500000,2019-01-01,46.500000,120.500000
9,78.500000,61.500000,10139.500000,2019-01-01,62.500000,120.500000
10,78.099998,61.099998,10139.099609,2019-01-01,62.099998,120.099998
...,...,...,...,...,...,...
28354,67.500000,33.500000,10235.500000,2018-12-31,31.500000,150.500000
28355,67.099998,33.099998,10235.099609,2018-12-31,31.100000,150.100006
28356,67.500000,33.500000,10219.500000,2018-12-31,36.500000,130.500000
28357,67.500000,44.500000,10211.500000,2018-12-31,46.500000,120.500000


### Compressing Dataframe
cutting down the dataframe so that each date only appear once:

In [13]:
df = df.groupby('date').agg({'temp': 'max', 
                         'dew_point':'mean', 
                         'pressure':'mean', 
                         'wind_speed': 'mean',
                         'wind_direction': 'mean'})
df

,temp,dew_point,pressure,wind_speed,wind_direction
date,,,,,
2018-01-09,44.500000,-39.663635,10191.572266,51.299999,249.481827
2018-01-10,33.500000,-29.933332,10306.099609,33.099998,167.100006
2018-01-11,83.500000,12.106896,10284.596680,28.355173,98.320686
2018-01-12,72.500000,35.599998,10218.670898,25.171429,64.671425
2018-01-13,100.500000,70.157143,10004.442383,63.157143,230.442856
...,...,...,...,...,...
2019-12-27,100.500000,38.710526,10273.658203,32.184212,103.552635
2019-12-28,100.500000,20.556000,10217.883789,38.924000,314.804016
2019-12-29,83.099998,-7.714286,10227.792969,36.685715,165.399994


In [14]:
df.to_parquet('../../Project 1/DataFrames/weather', compression='gzip') 